# Actor-critic
#### (simple mode)

![img](https://i.ytimg.com/vi/kgSylgBFi-I/maxresdefault.jpg)

In [1]:
%env THEANO_FLAGS='floatX=float32'
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
#     !bash ../xvfb start
    %env DISPLAY=:1

env: THEANO_FLAGS='floatX=float32'
env: DISPLAY=:1


In [3]:
import gym
import numpy as np
from itertools import count
from collections import namedtuple
import matplotlib.pyplot as plt
%matplotlib inline


In [4]:
env = gym.make("CartPole-v0").env
env.reset()
n_actions = env.action_space.n
state_dim = env.observation_space.shape

# plt.imshow(env.render("rgb_array"))

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [5]:
env.render(mode='rgb_array')

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [5]:
SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])

![img](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTjWnsh-kHzQGrARn5YxqHCgYgedJk5ZQCOHuh-ysdVpllb3g054A)

In [7]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(4, 128)
        
        self.action_head = nn.Linear(128, 2)
        self.value_head = nn.Linear(128, 1)

        self.saved_actions = []
        self.rewards = []

    def forward(self, x):
        <Требуется заимплементить>
        #Учтите, что актор возвращает нам полиси(распределение)
        #а критик значение V на текущем стейте

In [8]:
model = Policy()
optimizer = <ваш любимый оптимизатор>
eps = np.finfo(np.float32).eps.item()

In [9]:
def select_action(state):
    state = torch.from_numpy(state).float()
    probs, state_value = <Получить policy и V(s)>
    m = Categorical(probs)
    action = <Семплируем действие>
    model.saved_actions.append(SavedAction(m.log_prob(action), state_value))
    return action.item()


In [13]:
def finish_episode(gamma):
    R = 0
    saved_actions = model.saved_actions
    policy_losses = []
    value_losses = []
    rewards = []
    for r in model.rewards[::-1]:
        R = r + gamma * R
        rewards.insert(0, R)
    rewards = torch.tensor(rewards)
    rewards = (rewards - rewards.mean()) / (rewards.std() + eps)
    
    for (log_prob, value), r in zip(saved_actions, rewards):
        reward = r - value.item()
        policy_losses.append(<кажется вы знаете что сюда давать (вспомните reinforce)>)
        value_losses.append(F.smooth_l1_loss(value, torch.tensor([r])))
        
    optimizer.zero_grad()
    # Создайте лосс состоящий из двух частей
    loss = <>
    # можно отправляться назад
    loss.backward()
    optimizer.step()
    
    del model.rewards[:]
    del model.saved_actions[:]

In [ ]:
render = False # Надо поменять, если хочется картинок
log_interval = 500
running_reward = 10
gamma = 0.99

for i_episode in count(1):
    state = env.reset()
    for t in range(150):
        action = <выбрать действие>
        state, reward, done, _ = env.step(action)
        if render:
            env.render()
        model.rewards.append(reward)
        if done:
            break

    running_reward = running_reward * 0.99 + t * 0.01
    <Закончить эпизод>
    if i_episode % log_interval == 0:
        print('Episode {}\tLast length: {:5d}\tAverage length: {:.2f}'.format(
                i_episode, t, running_reward))
    if running_reward >= env.spec.reward_threshold:
        print("Победа! Задача решена с наградой {} !".format(running_reward))
        break